# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_to_load = "../output_data/cities.csv"

df_city_data = pd.read_csv(cities_data_to_load)
df_city_data = df_city_data.drop(columns=['Unnamed: 0'])
df_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kelapa Sawit,1.67,103.53,77.00,94,20,1.12,MY,1598811320
1,Gisborne,-38.65,178.00,43.00,92,0,3.00,NZ,1598811304
2,Mangan,27.52,88.53,76.24,59,98,0.67,IN,1598811320
3,Cascais,38.70,-9.42,72.00,46,0,27.51,PT,1598811180
4,Port Alfred,-33.59,26.89,52.00,68,32,12.06,ZA,1598811321
...,...,...,...,...,...,...,...,...,...
559,Zyryanovsk,49.74,84.27,53.74,71,0,1.54,KZ,1598811451
560,Rock Sound,24.90,-76.20,83.82,65,53,4.05,BS,1598811451
561,Chetwayi,10.53,76.05,82.40,78,40,6.93,IN,1598811452
562,Koslan,63.46,48.90,49.91,87,100,4.59,RU,1598811452


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

plot_locations = df_city_data[["Lat","Lng"]]
#print(plot_locations)
heat_map_weight = df_city_data['Humidity'].astype(float)
#print(heat_map_weight)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(plot_locations, weights=heat_map_weight, dissipating=False, 
                                 max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# Narrow down dataframe to my perfect weather

df_my_perfect_weather = df_city_data.loc[(df_city_data['Max Temp'] >= 89) &
                                         (df_city_data['Wind Speed'] <= 20) &
                                         (df_city_data['Cloudiness'] <= 50) &
                                         (df_city_data['Humidity'] <= 50)]

df_my_perfect_weather.dropna()
df_my_perfect_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,Las Vegas,36.17,-115.14,97.00,14,1,5.82,US,1598810730
111,Bilma,18.69,12.92,95.09,34,2,9.13,NE,1598811191
143,Vilhena,-12.74,-60.15,91.40,41,40,6.93,BR,1598811352
155,Saint George,37.10,-113.58,91.99,21,1,5.82,US,1598811069
160,Siverek,37.76,39.32,89.60,13,0,6.93,TR,1598811356
179,Awjilah,29.11,21.29,91.71,18,0,8.50,LY,1598811361
226,Urucará,-2.54,-57.76,97.21,38,36,3.42,BR,1598811373
261,Manicoré,-5.81,-61.30,98.73,30,42,3.49,BR,1598811381
313,Hays,30.05,-98.03,98.01,49,2,5.01,US,1598811393
350,Ţahţā,26.77,31.50,96.80,22,0,9.17,EG,1598811366


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = df_my_perfect_weather.loc[:,["City", "Country", "Lat", "Lng"]]
#hotel_df
hotel_df['Hotel Name']= ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
22,Las Vegas,US,36.17,-115.14,
111,Bilma,NE,18.69,12.92,
143,Vilhena,BR,-12.74,-60.15,
155,Saint George,US,37.10,-113.58,
160,Siverek,TR,37.76,39.32,
179,Awjilah,LY,29.11,21.29,
226,Urucará,BR,-2.54,-57.76,
261,Manicoré,BR,-5.81,-61.30,
313,Hays,US,30.05,-98.03,
350,Ţahţā,EG,26.77,31.50,


In [48]:
#Base URL for places nearby

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Search Paramaeters

target_search = "hotel"
target_radius = 5000
target_type = "lodging"

row_count = 0

print("---------------------------------")
print("- Searching for vacation hotels -")
print("---------------------------------")


for index, row in hotel_df.iterrows():
    
    #Parameter dictionary for API

    params = {
        "location": f"{row['Lat']},{row['Lng']}",
        "keywords": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[row_count, -1] = name
    row_count = row_count + 1

my_hotels = hotel_df
my_hotels

---------------------------------
- Searching for vacation hotels -
---------------------------------


,City,Country,Lat,Lng,Hotel Name
22,Las Vegas,US,36.17,-115.14,Plaza Hotel & Casino
111,Bilma,NE,18.69,12.92,Expeditions Ténére voyages
143,Vilhena,BR,-12.74,-60.15,Hotel Mirage
155,Saint George,US,37.10,-113.58,Best Western Plus Abbey Inn
160,Siverek,TR,37.76,39.32,Mercan Hotel
179,Awjilah,LY,29.11,21.29,
226,Urucará,BR,-2.54,-57.76,Pousada Maria Bonita
261,Manicoré,BR,-5.81,-61.30,
313,Hays,US,30.05,-98.03,Sinya on Lone Man Creek
350,Ţahţā,EG,26.77,31.50,Møhãmęd Tãręk 🏠


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))